In [1]:
pip install contractions

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install nlpaug

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 11.1 MB/s eta 0:00:0000:01


# Imports

In [3]:
# DataFrame
import pandas as pd

import string

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE

# Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data Augmentation
import nlpaug.augmenter.word as naw
from tqdm import tqdm
from sklearn.utils import shuffle

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Static variables

In [4]:
cleaning_regex = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

w2v_size = 300
w2v_window = 7
w2v_min_count = 10

In [ ]:
sequence_length = 300
epochs_number = 32
batch_size = 1024

In [5]:
# stop_words = stopwords.words("english")
# stop_words.remove('not')
stemmer = SnowballStemmer("english")

In [5]:
stop_words = ['a','an','the','and','but',
 'if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after',
 'above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there',
 'when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','only','own',
 'same','so','than','too','very','s','t','can','will','just',]

stop_words = set(stopwords.words('english')) 

# Read Dataset

In [6]:
df = pd.read_csv('/kaggle/input/toxicity-analysis/train.csv')

# Split the data into training and test sets with an 80:20 ratio
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Data Preprocessing

In [7]:
import contractions

def clean_text(text):
    text = contractions.fix(text)
    text = re.sub(cleaning_regex, ' ', str(text).lower()).strip()
    digit_remover = str.maketrans('', '', string.digits)
    text = text.translate(digit_remover)
    return text

def tokenize(text, stem=False):
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return tokens

def preprocess(text, stem=False):
    text = clean_text(text)
    tokens = tokenize(text, stem)
    return " ".join(tokens)

In [8]:
train_df.comment_text = train_df.comment_text.apply(lambda x: preprocess(x), True)
test_df.comment_text = test_df.comment_text.apply(lambda x: preprocess(x), True)

# Data Augmentation with word2vec

In [9]:
aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)

def augment_text(df,samples=5000,pr=0.2):
    aug.aug_p=pr
    new_text=[]
    df_n=df[df.toxic==1].reset_index(drop=True)
    for i in tqdm(np.random.randint(0,len(df_n),samples)):
        text = df_n.iloc[i]['comment_text']
        augmented_text = aug.augment(text)[0]
        new_text.append(augmented_text)

    new=pd.DataFrame({'comment_text':new_text,'toxic':1})
    df=shuffle(df.append(new).reset_index(drop=True))
    return df

In [10]:
train_df = augment_text(train_df)

100%|██████████| 5000/5000 [00:17<00:00, 289.46it/s]


In [12]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
85640,ba78c0a98872f3c6,website fancy flash cannot point direct source...,0,0.0,0.0,0.0,0.0,0.0
38521,9bb821e742a56d74,remarks talk pallar accurate articles area cou...,0,0.0,0.0,0.0,0.0,0.0
127807,NaN,idiot ever check item clause decide delete act...,1,NaN,NaN,NaN,NaN,NaN
115135,3180383b926cc0f6,ping hello months since last conversation want...,0,0.0,0.0,0.0,0.0,0.0
111457,a0b5699134fc5d6b,plainly stated abuse plan repurpose appropriat...,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
42848,ae54aa3267350ac1,fight argument also stingy retard spastic smel...,1,0.0,0.0,0.0,0.0,0.0
48597,89dcc5d47d9d8651,think edit warring settled,0,0.0,0.0,0.0,0.0,0.0
34096,631baa9cf650cb18,may something address bank reform act page how...,0,0.0,0.0,0.0,0.0,0.0
126802,0a00be3bc07532e2,welcome wikipedia welcome help create new cont...,0,0.0,0.0,0.0,0.0,0.0


In [13]:
documents = [_text.split() for _text in train_df.comment_text] 

In [14]:
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=w2v_size, window=wv2_window, min_count=w2v_min_count, 
                                            workers=8)

In [15]:
w2v_model.build_vocab(documents)

In [16]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(131535049, 144770144)

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.comment_text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 144296


In [18]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_df.comment_text), maxlen=sequence_length)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_df.comment_text), maxlen=sequence_length)

In [19]:
encoder = LabelEncoder()
encoder.fit(train_df['toxic'].tolist())

y_train = encoder.transform(train_df['toxic'].tolist())
y_test = encoder.transform(test_df['toxic'].tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

y_train (132656, 1)
y_test (31915, 1)


In [20]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (132656, 300)
y_train (132656, 1)

x_test (31915, 300)
y_test (31915, 1)


In [20]:
embedding_matrix = np.zeros((vocab_size, w2v_size))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(144296, 300)


In [21]:
embedding_layer = Embedding(vocab_size, w2v_size, weights=[embedding_matrix], input_length=sequence_length, trainable=False)

In [22]:
import keras
from keras.models import Sequential

In [23]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          43288800  
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 43,449,301
Trainable params: 160,501
Non-trainable params: 43,288,800
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [25]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [26]:
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=5,
                    validation_split=0.1,
                    verbose=10,
                    callbacks=callbacks)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [27]:
score = model.evaluate(x_test, y_test, batch_size=32)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

32/32 [==============================] - 3s 102ms/step - loss: 0.1119 - accuracy: 0.9570

ACCURACY: 0.9569794535636902
LOSS: 0.11192872375249863


In [28]:
from sklearn.metrics import confusion_matrix,classification_report

y_pred = model.predict(x_test)

998/998 [==============================] - 74s 74ms/step


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [29]:
y_pred_report = (y_pred > 0.5).astype(int)
print(classification_report(y_test,y_pred_report))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     28859
           1       0.76      0.81      0.78      3056

    accuracy                           0.96     31915
   macro avg       0.87      0.89      0.88     31915
weighted avg       0.96      0.96      0.96     31915



In [67]:
def predict(text, include_neutral=True):
    text = preprocess(text)
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=sequence_length)
    print(text)
    score = model.predict([x_test])[0]
    if score <= 0.5:
        print("Not an insult.")
    else:
        print("Insult")

In [34]:
predict("You are so beautiful, my eyes are just hypnotised")

beautiful eyes hypnotised
1/1 [==============================] - 0s 86ms/step
Not an insult.


In [35]:
predict("you are such a stupid arse")

stupid arse
1/1 [==============================] - 0s 87ms/step
Insult


In [36]:
predict("Fuck off, you dumb!!! I hate you so much.")

fuck dumb hate much
1/1 [==============================] - 0s 87ms/step
Insult


In [37]:
predict("Any chance you'll go out with me tonight?")

chance go tonight
1/1 [==============================] - 0s 87ms/step
Not an insult.


In [38]:
predict("you are a hero")

hero
1/1 [==============================] - 0s 87ms/step
Not an insult.


In [79]:
predict("fack you and your journal")

fack journal
1/1 [==============================] - 0s 96ms/step
Not an insult.


In [103]:
import nltk
from nltk.corpus import words
nltk.download('words')
word_list = set(words.words())  # Faster lookups with a set

def is_word(word):
    return word.lower() in word_list

def is_bad_word(word):
    bad_words = ["cunt", "fuck", "ass", "moron", "ugly", ""]
    return word.lower() in bad_words

def find_and_replace(sentence):
    sentence = sentence.split()
    for i, word in enumerate(sentence):
        found_insult = False
        found_word = None
        if not all(char.isalpha() or char.isspace() for char in word):
            for j, char in enumerate(word):
                if not char.isalpha():
                    for replacement in 'abcdefghijklmnopqrstuvwxyz':
                        possible_word = word[:j] + replacement + word[j+1:]
                        if is_bad_word(possible_word):
                            sentence[i] = possible_word
                            found_insult = True
                            break
                        if is_word(possible_word):
                            found_word = possible_word
        if found_insult is False and found_word is not None:
            sentence[i] = found_word
    return ' '.join(sentence)

# Example usage:
print(find_and_replace("you are a m0ron"))

you are a moron


[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [87]:
text = "fuck you and your journal, just fuck off this site"
def remove_non_letters(sentence):
    cleaned_sentence = ""
    for char in sentence:
        if char.isalpha() or char.isspace():
            cleaned_sentence += char
    return cleaned_sentence

print(remove_non_letters(text))

fuck you and your journal just fuck off this site


In [106]:
predict("you are a horrible person that doesn't deserve to live on this planet, just fuck off")

horrible person not deserve live planet fuck
1/1 [==============================] - 0s 88ms/step
Insult


In [91]:
import nltk
from nltk.corpus import wordnet

def negate_sentence(sentence):
    sentence = contractions.fix(sentence)
    negations = ['not', 'no', 'never', 'neither', 'nor']
    words = sentence.split()
    new_sentence = []
    negate_next = False
    
    for i, word in enumerate(words):
        if word in negations:
            negate_next = True
            negate_word = word
        elif negate_next:
            # Convert negated word to antonym
            antonyms = []
            for syn in wordnet.synsets(word):
                for lemma in syn.lemmas():
                    for antonym in lemma.antonyms():
                        antonyms.append(antonym.name())
            if antonyms:
                new_sentence.append(antonyms[0])
            else:
                new_sentence.append(negate_word)
                new_sentence.append(word)
            negate_next = False
        else:
            new_sentence.append(word)
            
    return ' '.join(new_sentence)

In [97]:
negate_sentence(preprocess("you are not an ugly person as everybody says"))

'beautiful person everybody says'

In [38]:
predict(negate_sentence("You're not moron, truth is not beyond your reach"))

1/1 [==============================] - 0s 54ms/step
[0.70553344]


In [98]:
predict("beautiful person everybody says")

beautiful person everybody says
1/1 [==============================] - 0s 89ms/step
Not an insult.


In [31]:
def save_data(data, pathfile):
    f = open(pathfile, 'wb')
    pickle.dump(data, f)
    f.close()

In [33]:
save_data(model, '/kaggle/working/LSTM_Word2Vec_v2.pkl')

In [99]:
model.save_weights('insult_model_weights.h5')

In [100]:
import pickle
with open('tokenizer_insult.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)